In [15]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')


# Data
Every time a user opens a mobile app, an auction is going on behind the scenes. The highest bidder gets to advertise his ad to the user.
## Auctions Table

In [16]:
sql_query = 'SELECT * FROM auctions;'
with engine.connect() as db_con:
    Auctions_table = pd.read_sql(sql_query, con=db_con)

Auctions_table

,id,eventTimestamp,unitDisplayType,brandName,bundleId,countryCode,deviceId,osAndVersion,bidFloorPrice,sentPrice
0,1,1657758857892,rewarded,Generic,com.loop.match3d,US,6a0a94554cf,Android-4.0,0.05,0.06
1,2,1657210707978,interstitial,Generic,com.loop.match3d,ZA,6a0b0e59f45,Android-4.0,0.01,0.16
2,3,1657392939412,interstitial,Motorola,com.YayySAL.DodgeAgent,BR,6a0fa820c46,Android-10.0,0.05,0.06
3,4,1657386816882,interstitial,Huawei,com.tintash.nailsalon,IQ,6a142bdbea2,Android-9.0,0.01,0.05
4,5,1657211600823,banner,Generic,com.tilegarden.match3,US,6a16943a771,Android-4.0,0.01,0.03
...,...,...,...,...,...,...,...,...,...,...
207915,207916,1657731605063,banner,Generic,com.tilegarden.match3,NZ,80edf5837e8,Android-4.0,0.01,0.02
207916,207917,1657657131101,interstitial,Apple,1436213906,VN,80ee6a5eebf,iOS-13.6,0.01,0.90
207917,207918,1657156070854,rewarded,Generic,com.loop.match3d,US,80efe531afc,Android-4.0,0.05,0.69
207918,207919,1657659603568,banner,Apple,1502447854,US,80f02dd397f,iOS-15.0,0.01,0.10


## App Vectors table
We've gathered the first few sentences from the app store description and embedded it with a [model](https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1)

In [17]:
sql_query = f'''
SELECT
    *
FROM app_vectors
'''
has_embedding = False
while not has_embedding:
    with engine.connect() as db_con:
        app_vectors = pd.read_sql(sql_query, con=db_con)
    has_embedding = (~app_vectors["embedding"].isna()).all()
    if not has_embedding:
        print("Waiting for embeddings...")
        time.sleep(15)

app_vectors


,id,bundleId,content,embedding
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.31683633,0.6250093,-0.120355725,0.2174227,-..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.54112935,0.7446765,-0.58174425,0.9596788,0...."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176743,0.60594726,-1.1395669,0.07631005,-..."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.050924387,1.0967343,-0.30831495,0.13220492,..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.8975629,0.46442613,-0.18019181,1.2617054,0...."
...,...,...,...,...
85,84,1502447854,"Get ready for a new, challenging and original ...","[0.29414183,0.6344754,-0.038190607,0.21263401,..."
86,85,1436213906,Gather your riches and travel across the open ...,"[0.8503366,0.3483968,-0.6132452,0.55295014,0.0..."
87,86,1542256628,"Your mission, should you choose to accept it, ...","[0.54112935,0.7446765,-0.58174425,0.9596788,0...."
88,87,1579489488,Become the hero we need in this tangled puzzle...,"[0.51124793,0.5128076,-0.056303732,1.2819327,0..."


We can use the `<=>` operator to run vector search within the database

In [18]:

vec = json.loads(app_vectors.embedding[0]) # get the first embedding
print ("Embedding size: {l}".format(l=len(vec)))

sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

Embedding size: 1024


,bundleId
0,com.loop.match3d
1,com.loop.match3d
2,com.loop.match3d
3,com.loop.match3d
4,com.loop.match3d
...,...
85,com.tintash.nailsalon
86,1523081624
87,com.tintash.nailsalon
88,1523081624


# What you need to do
## The hypothesis
We assume that apps with similar desciptions, would have a similar asking price in the auctions (`sentPrice` column).

Use cosine similarity (`<=>`) on the embeddings to find similar apps, and any statistical tools you find suitable to prove or disprove this hypothesis.

## Is it consistent?
There are several other features in the auctions table (such as `CountryCode` and `OS`), 
Do your findings hold for those as well?

In [19]:
!pip install scikit-learn

In [20]:
import ast
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
import numpy as np

In [21]:
print(Auctions_table['bundleId'].unique())
print(Auctions_table['bundleId'].nunique())

['com.loop.match3d' 'com.YayySAL.DodgeAgent' 'com.tintash.nailsalon'
 'com.tilegarden.match3' 'com.kamilbilge.ropesavior3d' '1529614832'
 '1523081624' '1502447854' 'com.AppIdeas.LevelUpRunner' 'se.ace.fishinc'
 '1436213906' '1542256628' 'com.volt.dresstoimpress' '1579489488'
 'dovi.coalmining.inc' '1586795332' '1582745578' '1569586264']
18


In [22]:
print(app_vectors['bundleId'].unique())
print(app_vectors['bundleId'].nunique())

['com.loop.match3d' 'com.YayySAL.DodgeAgent' 'com.tintash.nailsalon'
 'com.tilegarden.match3' 'com.AppIdeas.LevelUpRunner' '1582745578'
 '1569586264' 'com.kamilbilge.ropesavior3d' 'se.ace.fishinc'
 'com.volt.dresstoimpress' 'dovi.coalmining.inc' '1529614832' '1523081624'
 '1502447854' '1436213906' '1542256628' '1579489488' '1586795332']
18


In [23]:
# Converting embedding lists to lists of floats instead of string

app_vectors['embedding'] = app_vectors['embedding'].apply(ast.literal_eval)

In [24]:
# creating embedding distance for all bundles

bundles = app_vectors['bundleId'].unique()
embedding_distances = {}

for i, bundle1 in enumerate(bundles):
    distances_for_bundle = {}
    embeddings1 = app_vectors[app_vectors['bundleId'] == bundle1]['embedding'].values[0]
    for j, bundle2 in enumerate(bundles):
        if i != j:
            embeddings2 = app_vectors[app_vectors['bundleId'] == bundle2]['embedding'].values[0]
            similarity = cosine_similarity([embeddings1], [embeddings2])[0][0]
            distances_for_bundle[bundle2] = similarity

    distances_for_bundle = dict(sorted(distances_for_bundle.items(), key=lambda item: item[1]))
    embedding_distances[bundle1] = distances_for_bundle

In [25]:
embedding_distances

{'com.loop.match3d': {'com.tintash.nailsalon': 0.45934304621280264,
  '1523081624': 0.45934304621280264,
  '1582745578': 0.46973979905376395,
  'dovi.coalmining.inc': 0.4697398662939984,
  'com.YayySAL.DodgeAgent': 0.4809743967810113,
  '1542256628': 0.4809743967810113,
  '1569586264': 0.5018004362919362,
  'com.volt.dresstoimpress': 0.5018004929426269,
  'se.ace.fishinc': 0.5029227178430007,
  'com.AppIdeas.LevelUpRunner': 0.5096486705809213,
  '1586795332': 0.5096486705809213,
  '1436213906': 0.5303715803439515,
  'com.kamilbilge.ropesavior3d': 0.5578786969812164,
  '1579489488': 0.5578786969812164,
  'com.tilegarden.match3': 0.6586367713547914,
  '1529614832': 0.719057488959124,
  '1502447854': 0.9226371026478282},
 'com.YayySAL.DodgeAgent': {'com.tintash.nailsalon': 0.38917027905933965,
  '1523081624': 0.38917027905933965,
  '1569586264': 0.43780194221441593,
  'com.volt.dresstoimpress': 0.4378019706771192,
  '1529614832': 0.4443962221483879,
  'com.tilegarden.match3': 0.4469381497

In [26]:
Auctions_table['sentPrice']

0         0.06
1         0.16
2         0.06
3         0.05
4         0.03
          ... 
207915    0.02
207916    0.90
207917    0.69
207918    0.10
207919    0.77
Name: sentPrice, Length: 207920, dtype: float64

In [27]:
Auctions_table.columns

Index(['id', 'eventTimestamp', 'unitDisplayType', 'brandName', 'bundleId',
       'countryCode', 'deviceId', 'osAndVersion', 'bidFloorPrice',
       'sentPrice'],
      dtype='object')

In [30]:
pairwise_distances_data = []

# Populate the list with pairwise distances
for bundle1, distances in embedding_distances.items():
    for bundle2, distance in distances.items():
        pairwise_distances_data.append({'bundle1': bundle1, 'bundle2': bundle2, 'distance': distance})

# Create a DataFrame from the list of dictionaries
pairwise_distances = pd.DataFrame(pairwise_distances_data)

# Assuming 'Auctions_table' is your pandas DataFrame
# Merge 'pairwise_distances' DataFrame with 'Auctions_table' to get 'sentPrice' for each bundle
merged_df = pd.merge(pairwise_distances, Auctions_table[['bundleId', 'sentPrice']], how='inner', left_on='bundle1', right_on='bundleId')
merged_df.rename(columns={'sentPrice': 'sentPrice1'}, inplace=True)
merged_df.drop(columns=['bundleId'], inplace=True)

merged1_df = pd.merge(pairwise_distances, Auctions_table[['bundleId', 'sentPrice']], how='inner', left_on='bundle2', right_on='bundleId')
merged1_df.rename(columns={'sentPrice': 'sentPrice2'}, inplace=True)
merged1_df.drop(columns=['bundleId'], inplace=True)

# Calculate correlation between distances and 'sentPrice'
# correlation = merged_df['distance'].corr(merged_df['sentPrice'])

# print("Correlation between embedding distances and sentPrice:", correlation)

In [36]:
merged_df

,bundle1,bundle2,distance,sentPrice1
0,com.loop.match3d,com.tintash.nailsalon,0.459343,0.06
1,com.loop.match3d,com.tintash.nailsalon,0.459343,0.16
2,com.loop.match3d,com.tintash.nailsalon,0.459343,0.24
3,com.loop.match3d,com.tintash.nailsalon,0.459343,0.02
4,com.loop.match3d,com.tintash.nailsalon,0.459343,0.30
...,...,...,...,...
3534635,1586795332,com.AppIdeas.LevelUpRunner,1.000000,0.19
3534636,1586795332,com.AppIdeas.LevelUpRunner,1.000000,0.64
3534637,1586795332,com.AppIdeas.LevelUpRunner,1.000000,7.96
3534638,1586795332,com.AppIdeas.LevelUpRunner,1.000000,0.52


In [35]:
merged1_df

,bundle1,bundle2,distance,sentPrice2
0,com.loop.match3d,com.tintash.nailsalon,0.459343,0.05
1,com.loop.match3d,com.tintash.nailsalon,0.459343,0.04
2,com.loop.match3d,com.tintash.nailsalon,0.459343,0.03
3,com.loop.match3d,com.tintash.nailsalon,0.459343,0.04
4,com.loop.match3d,com.tintash.nailsalon,0.459343,0.06
...,...,...,...,...
3534635,1586795332,com.AppIdeas.LevelUpRunner,1.000000,0.06
3534636,1586795332,com.AppIdeas.LevelUpRunner,1.000000,0.05
3534637,1586795332,com.AppIdeas.LevelUpRunner,1.000000,0.10
3534638,1586795332,com.AppIdeas.LevelUpRunner,1.000000,0.18


In [37]:
df = pd.merge(merged_df, merged1_df[['sentPrice2']], how='inner', left_on='bundle1', right_on='bundle2') 

KeyError: 'bundle2'